In [1]:
import os
import os.path as osp
import itertools
import functools
import operator
from glob import glob
from shutil import copy as copy_
from collections import namedtuple

import numpy as np
from skimage.io import imread, imsave
from tqdm import tqdm

In [2]:
COPY_MODE = 'LINK'
PALETTE = [
    [222, 211, 140],
    [130, 57,  53 ],
    [0,   0,   0  ],
    [69, 137,  148]
]

In [3]:
def visualize_diff(cm, gt):
    if isinstance(cm, str):
        cm = imread(cm)
    if isinstance(gt, str):
        gt = imread(gt)

    assert cm.shape == gt.shape and cm.dtype == gt.dtype == np.uint8

    cm = (cm>127).astype(np.bool)
    gt = (gt>127).astype(np.bool)

    vis_map = np.zeros((*cm.shape, 3), dtype='uint8')

    tp_mask = cm & gt
    fp_mask = cm & ~gt
    tn_mask = ~cm & ~gt
    fn_mask = ~cm & gt

    vis_map[tp_mask] = PALETTE[0]
    vis_map[fp_mask] = PALETTE[1]
    vis_map[tn_mask] = PALETTE[2]
    vis_map[fn_mask] = PALETTE[3]

    return vis_map

def copy(src, dst):
    dst_dir = osp.dirname(dst)
    if not osp.exists(dst_dir):
        os.makedirs(dst_dir)
    if COPY_MODE == 'LINK':
        if osp.exists(dst):
            os.remove(dst)
        os.symlink(src, dst)
    else:    
        copy_(src, dst)

def collect(out_dir, gt_dir, cm_dirs_dict, gen_vis_map=False, ext='.png'):
    _CM_EXT = '.png'

    def _flatten(lst):
        for l in lst:
            yield from l

    class _Item:
        def __init__(self, tag, src, stem, ext, gt_path=None, out_dir='.'):
            self.tag = tag
            self.src = src
            self.stem = stem
            self.ext = ext
            self.gt_path = gt_path
            self.out_dir = out_dir

        @property
        def dst(self):
            return osp.join(self.out_dir, self.stem, self.tag+self.ext)

        def set_out_dir(self, out_dir):
            self.out_dir = out_dir

    gt_paths = glob(osp.join(gt_dir, '*'+ext))
    stems = [osp.splitext(osp.basename(p))[0] for p in gt_paths]
    gt_items = list(map(functools.partial(_Item, 'gt'), gt_paths, stems, itertools.cycle([ext]), gt_paths))
    
    cm_items_list = []
    for tag, cm_dir in cm_dirs_dict.items():
        cm_paths = (osp.join(cm_dir, stem+_CM_EXT) for stem in stems)
        cm_items_list.append(list(map(functools.partial(_Item, tag), cm_paths, stems, itertools.cycle([_CM_EXT]), gt_paths)))

    for item in tqdm(itertools.chain(gt_items, _flatten(cm_items_list))):
        item.set_out_dir(out_dir)
        copy(item.src, item.dst)
        if gen_vis_map:
            vis_map = visualize_diff(item.src, item.gt_path)
            imsave(item.dst[:-len(item.ext)]+'_vis'+item.ext, vis_map, check_contrast=False)

In [4]:
# SVCD Dataset
collect(
    '/home/gdf/code/_CDLab/exp/svcd/out/collect/', 
    '/home/gdf/data/LEVIR-CD/ori/test/label/',
    {
        'cdnet': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_28/cdnet/',
        'unet': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_26/unet/',
        'siamunet-conc': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_20/siamunet-conc/',
        'siamunet-diff': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_19/siamunet-diff/',
        'stanet': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_28/stanet/',
        'lunet': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_29/lunet/',
        'snunet': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_28/snunet/',
        'bit': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_22/bit/',
        'ifn': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_29/ifn/',
        'p2v': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_29/p2v/',
        'p2v_2donly': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_27/p2v_2donly/',
        'p2v_decouple': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_27/p2v_decouple/',
        'p2v_notemporal': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_28/p2v_notemporal/',
        'p2v_late_fusion': '/home/gdf/code/_CDLab/exp/svcd/out/epoch_29/p2v_latefusion/',
    },
    gen_vis_map=True,
    ext='.jpg'
)

13015it [05:42, 23.19it/s]

In [ ]:
# LEVIR-CD Dataset
collect(
    '/home/gdf/code/_CDLab/exp/levircd/out/collect/', 
    '/home/gdf/data/HR/ChangeDetectionDataset/Real/subset/test/OUT',
    {
        'cdnet': '/home/gdf/code/_CDLab/exp/levircd/sw_out/cdnet/',
        'unet': '/home/gdf/code/_CDLab/exp/levircd/sw_out/unet/',
        'siamunet-conc': '/home/gdf/code/_CDLab/exp/levircd/sw_out/siamunet-conc/',
        'siamunet-diff': '/home/gdf/code/_CDLab/exp/levircd/sw_out/siamunet-diff/',
        'stanet': '/home/gdf/code/_CDLab/exp/levircd/sw_out/stanet/',
        'lunet': '/home/gdf/code/_CDLab/exp/levircd/sw_out/lunet/',
        'snunet': '/home/gdf/code/_CDLab/exp/levircd/sw_out/snunet/',
        'bit': '/home/gdf/code/_CDLab/exp/levircd/sw_out/bit/',
        'ifn': '/home/gdf/code/_CDLab/exp/levircd/sw_out/ifn/',
    },
    gen_vis_map=True,
    ext='.png'
)

In [ ]:
# WHU Dataset
collect(
    '/home/gdf/code/_CDLab/exp/whu/out/collect/', 
    '/home/gdf/data/WHU/256x256_random/test/label/',
    {
        'cdnet': '/home/gdf/code/_CDLab/exp/whu/out/epoch_29/cdnet/',
        'unet': '/home/gdf/code/_CDLab/exp/whu/out/epoch_27/unet/',
        'siamunet-conc': '/home/gdf/code/_CDLab/exp/whu/out/epoch_27/siamunet-conc/',
        'siamunet-diff': '/home/gdf/code/_CDLab/exp/whu/out/epoch_28/siamunet-diff/',
        'stanet': '/home/gdf/code/_CDLab/exp/whu/out/epoch_27/stanet/',
        'lunet': '/home/gdf/code/_CDLab/exp/whu/out/epoch_28/lunet/',
        'snunet': '/home/gdf/code/_CDLab/exp/whu/out/epoch_24/snunet/',
        'bit': '/home/gdf/code/_CDLab/exp/whu/out/epoch_24/bit/',
        'ifn': '/home/gdf/code/_CDLab/exp/whu/out/epoch_10/ifn/',
        'p2v': '/home/gdf/code/_CDLab/exp/whu/out/epoch_29/p2v/',
    },
    gen_vis_map=True,
    ext='.png'
)